In [1]:
import lzma
import json
import pandas as pd
import numpy as np
import re
import glob
import os
import sys
import multiprocessing as mp
import pickle

In [2]:
os.chdir("data\loc_data_hpc")

In [3]:
os.getcwd()

'c:\\Users\\erica\\OneDrive - University of Southern California\\Projects_at_USC\\CKIDS_current\\data\\loc_data_hpc'

In [4]:
# lists of file names
# json xz
json_files = glob.glob("*_UTC.json")
# descrip txts
txt_files = glob.glob("*_UTC.txt")
# location txts
loc_files = glob.glob("*_location.txt")
# comments jsons
cmt_files = glob.glob("*_comments.json")


print(len(json_files))
print(len(txt_files))
print(len(loc_files))
print(len(cmt_files))

1588
1216
1243
1053


In [6]:
# num of jpgs?
jpg_files = glob.glob("*.jpg")
print(len(jpg_files))

2095


In [7]:
# Add all json files to a dataframe
temp = pd.DataFrame()

path_to_json = 'data\loc_data_hpc' 

json_pattern = os.path.join(path_to_json,'*_UTC.json')
file_list = glob.glob(json_pattern)

for file in json_files:
    # print(file)
    data = pd.read_json(file, orient=str)
    postnode = data['node']
    temp = temp.append(postnode, ignore_index = True)

In [8]:
temp.shape

(1588, 50)

In [9]:
temp.head(3)

,__typename,accessibility_caption,caption_is_edited,commenting_disabled_for_viewer,comments_disabled,dimensions,display_resources,display_url,edge_liked_by,edge_media_preview_comment,...,viewer_in_photo_of_you,dash_info,encoding_status,is_published,product_type,title,video_duration,video_url,video_view_count,edge_sidecar_to_children
0,GraphImage,Photo by Amanda Berkowitz in Maccheroni Republ...,0.0,0.0,0.0,"{'height': 1350, 'width': 1080}","[{'config_height': 800, 'config_width': 640, '...",https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 17},"{'count': 0, 'edges': []}",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GraphVideo,None,1.0,0.0,0.0,"{'height': 937, 'width': 750}","[{'config_height': 800, 'config_width': 640, '...",https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 234},"{'count': 28, 'edges': [{'node': {'created_at'...",...,0.0,"{'is_dash_eligible': True, 'number_of_qualitie...",None,1.0,feed,,18.833,https://scontent-lax3-1.cdninstagram.com/v/t50...,2740.0,NaN
2,GraphImage,"Photo by Happy in Drago Centro with @mpsiox, @...",0.0,0.0,0.0,"{'height': 1080, 'width': 1080}","[{'config_height': 640, 'config_width': 640, '...",https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 49},"{'count': 3, 'edges': [{'node': {'created_at':...",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
json_fnames = [f.split('.json')[0] for f in json_files]
json_fnamesS = pd.Series(json_fnames)
temp['file_name'] = json_fnamesS

In [11]:
temp2 = temp[temp['__typename'] == "GraphImage"]
print(temp2.shape)

(1106, 51)


In [12]:
df = temp2[['file_name', 'accessibility_caption', 'display_url', 'edge_liked_by', 'id', 'is_ad', 'location', 'owner', 'taken_at_timestamp']]

In [13]:
df.head()

,file_name,accessibility_caption,display_url,edge_liked_by,id,is_ad,location,owner,taken_at_timestamp
0,2020-03-01_00-09-37_UTC,Photo by Amanda Berkowitz in Maccheroni Republ...,https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 17},2254869203465042645,0.0,"{'has_public_page': True, 'id': 58466590, 'lat...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583021e+09
2,2020-03-01_01-07-11_UTC,"Photo by Happy in Drago Centro with @mpsiox, @...",https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 49},2254898176307906834,0.0,"{'has_public_page': True, 'id': 219692297, 'la...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583025e+09
3,2020-03-01_01-31-03_UTC,Photo by Redbird in Redbird. Image may contain...,https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 71},2254910189599787551,0.0,"{'has_public_page': True, 'id': 513097043, 'la...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583026e+09
4,2020-03-01_01-44-20_UTC,Photo by The Coolers in The Redwood Bar with @...,https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 45},2254916876981567398,0.0,"{'has_public_page': True, 'id': 677627, 'lat':...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583027e+09
5,2020-03-01_01-46-16_UTC,Photo by DRAGO Centro in Drago Centro. Image m...,https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 46},2254917851620553674,0.0,"{'has_public_page': True, 'id': 219692297, 'la...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583027e+09


In [14]:
df.to_csv("post_jsons_df_hpc.csv")

In [40]:
# Read in text descrip files (hashtags)

In [15]:
def read_txt(file):
    text = open(file, "r", encoding="utf8").read()
    return text   # return as string


def get_ht(text):
    ht = re.findall(r"#(\w+)", text)
    return ht

In [16]:
texts = []

for f in txt_files:
    text = open(f, 'r', encoding="utf8").read()
    texts.append(text) 

In [17]:
texts = list(map(read_txt, txt_files))
hashtags = list(map(get_ht, texts))

In [18]:
# get file names
txt_fnames = [f.split('.txt')[0] for f in txt_files]

# convert all to series, and concat them to a df
textsS = pd.Series(texts)
hashtagsS = pd.Series(hashtags)
txt_fnamesS = pd.Series(txt_fnames)

df_txt = pd.DataFrame(pd.concat([txt_fnamesS, textsS, hashtagsS], axis=1))
df_txt.columns = ['file_name', 'descrip', 'hashtags']

In [19]:
df_txt.to_csv("df_txt_hpc.csv")

In [20]:
# Join the json df and txt df by their common filename

In [21]:
print(df_txt.columns)
print(df.columns)

Index(['file_name', 'descrip', 'hashtags'], dtype='object')
Index(['file_name', 'accessibility_caption', 'display_url', 'edge_liked_by',
       'id', 'is_ad', 'location', 'owner', 'taken_at_timestamp'],
      dtype='object')


In [22]:
df_txt['file_name'].head()

0    2020-03-01_00-43-27_UTC
1    2020-03-01_01-31-03_UTC
2    2020-03-01_01-47-51_UTC
3    2020-03-01_02-58-19_UTC
4    2020-03-01_03-17-28_UTC
Name: file_name, dtype: object

In [23]:
df['file_name'].head

<bound method NDFrame.head of 0       2020-03-01_00-09-37_UTC
2       2020-03-01_01-07-11_UTC
3       2020-03-01_01-31-03_UTC
4       2020-03-01_01-44-20_UTC
5       2020-03-01_01-46-16_UTC
                 ...           
1581    2020-03-26_21-31-22_UTC
1582    2020-03-26_21-36-44_UTC
1584    2020-03-26_22-22-54_UTC
1585    2020-03-26_22-30-54_UTC
1587    2020-03-26_23-58-53_UTC
Name: file_name, Length: 1106, dtype: object>

In [24]:
# merge the post metadata (jsons) df and text descrip df (txt)
merged = pd.merge(df, df_txt, on='file_name')
print(merged.shape)

(821, 11)


In [25]:
merged.head()

,file_name,accessibility_caption,display_url,edge_liked_by,id,is_ad,location,owner,taken_at_timestamp,descrip,hashtags
0,2020-03-01_01-31-03_UTC,Photo by Redbird in Redbird. Image may contain...,https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 71},2254910189599787551,0.0,"{'has_public_page': True, 'id': 513097043, 'la...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583026e+09,"Wedding guests, by @KatiePritchard. Visit prof...","[RedbirdLA, DTLAWedding, weddingdog]"
1,2020-03-01_01-47-51_UTC,Photo by Josie Diaz Trunnell in Redbird with @...,https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 13},2254918644671070750,0.0,"{'has_public_page': True, 'id': 513097043, 'la...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583027e+09,Trunnell date night!\n,[]
2,2020-03-01_02-58-19_UTC,Photo by 𝗘𝗻𝗴𝗹𝗶𝘀𝗵𝗺𝗮𝗻 & 𝗰𝗼𝗿𝗴𝗶 in FIGat7th. Image...,https://scontent-lax3-2.cdninstagram.com/v/t51...,{'count': 33},2254954116237586018,0.0,"{'has_public_page': True, 'id': 4817825, 'lat'...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583031e+09,Green grass \n#Corgi #CorgiAddict #CorgiCommun...,"[Corgi, CorgiAddict, CorgiCommunity, コーギー, 코기,..."
3,2020-03-01_03-34-54_UTC,Photo by James J. Ackerman in Redbird. Image m...,https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 12},2254972524022866922,0.0,"{'has_public_page': True, 'id': 513097043, 'la...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583034e+09,Merrill Rodin at @redbirdla with @marthaackerm...,"[perspective, artinlife, downtown, losangeles_la]"
4,2020-03-01_04-47-43_UTC,Photo by Scott Olson in FIGat7th. Image may co...,https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 14},2255009173700278683,0.0,"{'has_public_page': True, 'id': 4817825, 'lat'...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.583038e+09,Somebody call the LAFD! 🔥\n,[]


In [26]:
merged.to_csv("posts_merged_df_hpc.csv")